# Access STAC items with PySTAC

[PySTAC](https://pystac.readthedocs.io/en/stable/) is a library for working with [STAC](https://docs.up42.com/developers/api-assets/stac-about). This tutorial explains how to get data items associated with your orders using PySTAC and visualize them on a map.

## Step 1. Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade rasterio geopandas matplotlib contextily -q

import pathlib

import up42

import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

# Print the path to the file
print(f"Credentials file is located at: {credentials_file_path}")

1. Click the link above to the created file and paste the following code:
    ```
    {
    "username": "<your-email-address>",
    "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for `username` and `password`.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

### 3. Create a PySTAC client connection

In [ ]:
UP42_client = up42.stac_client()

## Step 2. Search for STAC items

You can make a [detailed search request](https://docs.up42.com/developers/api-stac#detailed-search) to find specific STAC items in your storage.

### Search by tags

In [ ]:
# This filter searches for STAC items that contain "SPOT" and "Orthorectified" tags
filter = {
    "op": "a_contains",
    "args": [
        {"property": "tags"},
        ["SPOT", "Orthorectified"],
    ],
}

# Perform the search
stac_items_search = UP42_client.search(filter=filter)

# Iterate through each STAC item and print selected details
for stac_item in stac_items_search.items():
    print(f"STAC item ID:        {stac_item.id}")
    print(f"Tags:                {stac_item.properties['up42-user:tags']}\n")

### Search by order ID

In [ ]:
# Define an order ID
order_id = "68567134-27ad-7bd7-4b65-d61adb11fc78"

# This filter searches for STAC items associated with the defined order ID
filter = {
    "op": "=",
    "args": [
        {"property": "order_id"},
        order_id,
    ],
}

# Perform the search
stac_items_search = UP42_client.search(filter=filter)

# Iterate through each STAC item and print selected details
for stac_item in stac_items_search.items():
    print(f"STAC item ID:        {stac_item.id}")
    print(f"Order ID:            {stac_item.properties['up42-order:id']}\n")

### Search by date

In [ ]:
# This filter searches for STAC items acquired between March 21, 2023, and April 18, 2023
filter = {
    "op": "t_overlaps",
    "args": [
      {
        "property": "datetime"
      },
      {
        "interval": ["2023-03-21T00:00:00.000Z", "2023-04-18T00:00:00.000Z"]
      }
    ]
  }

# Perform the search
stac_items_search = UP42_client.search(filter=filter)

# Iterate through each STAC item and print selected details
for stac_item in stac_items_search.items():
    print(f"STAC item ID:        {stac_item.id}")
    print(f"Acquisition date:    {stac_item.properties['datetime']}\n")


### Search by cloud coverage

In [ ]:
# This filter searches for STAC items with cloud coverage between 4% and 5%
filter = {
    "op": "and",
    "args": [
      {
        "op": ">=",
        "args": [
          {
            "property": "eo:cloud_cover"
          },
          4.0
        ]
      },
      {
        "op": "<=",
        "args": [
          {
            "property": "eo:cloud_cover"
          },
          5.0
        ]
      }
    ]
  }

# Perform the search
stac_items_search = UP42_client.search(filter=filter)

# Iterate through each STAC item and print selected details
for stac_item in stac_items_search.items():
    print(f"STAC item ID:      {stac_item.id}")
    print(f"Cloud coverage:    {stac_item.properties['eo:cloud_cover']}\n")


## Step 3. Download and visualize results

### 1. Download results

Select desired bands and download respective asset.

In [ ]:
from pathlib import Path

selected_bands = {"red", "green", "blue"}
assets_with_bands = []
for asset in stac_item.assets.values():
    bands = asset.extra_fields.get("eo:bands", None)
    if not bands:
        continue
    band_names = [band.get("name") for band in bands]    
    if selected_bands.issubset(set(band_names)):
        matching_asset = asset
        break

rgb_file = matching_asset.file
local_rgb = rgb_file.download(output_directory=Path.home()) # download asset to your home directory

### 2. Visualize results

To visualize results on a map, retrieve the TIF files paths from the asset folder.

In [ ]:
aoi_geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [ -74.8500, 10.950 ],
            [ -74.8500, 11.050 ],
            [ -74.7900, 11.050 ],
            [ -74.7900, 10.950 ],
            [ -74.8500, 10.950 ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

gdf = gpd.GeoDataFrame.from_features(aoi_geojson)
gdf = gdf.set_crs('EPSG:4326')
gdf_proj = gdf.to_crs(epsg=stac_item.properties["proj:code"])

f, ax = plt.subplots(1, 1)
with rasterio.open(local_rgb) as src:
    gdf_proj.plot(ax=ax, alpha=0.5) # AOI used for search
    ctx.add_basemap(ax, crs=stac_item.properties["proj:code"], source=ctx.providers.CartoDB.Positron)
    rasterio.plot.show(src, ax=ax, alpha=0.6) # Resulting asset from the search
    plt.show()

# Read raster file with rasterio
raster = rasterio.open(rgb_file)
rasterio.plot.show(raster)